In [ ]:
import dotenv
import pendulum
from sqlalchemy import asc, cast
from sqlalchemy import create_engine, select, BigInteger, MetaData, Table
from sqlalchemy.orm import sessionmaker
from config import Settings
from models import MessageSql
from gridflo.asl.types import FloParamsHouse0
from gridflo.dijkstra_types import DNode, DEdge
from gridflo import Flo, DGraphVisualizer, DNodeVisualizer
from gridflo.asl.types import WinterOakSupergraphParams
from gridflo.supergraph_generator import WinterOakSupergraphParams
from gridflo.dijkstra_types import DNode
import gc
import os
import shutil
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from PIL import Image

print("\nWelcome to the FLO report generator!\n")
house_alias = input("Enter house alias: ")
if not house_alias:
    print("House alias is required")
    exit()

now = pendulum.now(tz='America/New_York')
yesterday_8pm = now.subtract(days=1).set(hour=20, minute=0, second=0, microsecond=0)

start_input = input("Enter start year, month, day, hour (default: yesterday 8pm): ")
if start_input and len(start_input.split(',')) == 4:
    START_YEAR, START_MONTH, START_DAY, START_HOUR = start_input.split(',')
    START_YEAR, START_MONTH, START_DAY, START_HOUR = int(START_YEAR), int(START_MONTH), int(START_DAY), int(START_HOUR)
else:
    START_YEAR, START_MONTH, START_DAY, START_HOUR = yesterday_8pm.year, yesterday_8pm.month, yesterday_8pm.day, yesterday_8pm.hour

end_input = input("Enter end year, month, day, hour (default: now): ")
if end_input and len(end_input.split(',')) == 4:
    END_YEAR, END_MONTH, END_DAY, END_HOUR = end_input.split(',')
    END_YEAR, END_MONTH, END_DAY, END_HOUR = int(END_YEAR), int(END_MONTH), int(END_DAY), int(END_HOUR)
else:
    END_YEAR, END_MONTH, END_DAY, END_HOUR = now.year, now.month, now.day, now.hour

start_time = pendulum.datetime(START_YEAR, START_MONTH, START_DAY, START_HOUR, tz='America/New_York')
end_time = pendulum.datetime(END_YEAR, END_MONTH, END_DAY, END_HOUR, tz='America/New_York')

# TEMPORARY
# house_alias = "oak"
# start_time = pendulum.datetime(2026, 1, 20, 20, tz='America/New_York')
# end_time = pendulum.datetime(2026, 1, 20, 22, tz='America/New_York')

start_ms = start_time.timestamp()*1000
end_ms = end_time.timestamp()*1000
print(f"Generating report for {house_alias} from {start_time} to {end_time}\n")

# ---------------------------------------------------
# Part 1: Find FLO params messages
# ---------------------------------------------------

stmt = select(MessageSql).filter(
    MessageSql.message_type_name == "flo.params.house0",
    MessageSql.from_alias == f"hw1.isone.me.versant.keene.{house_alias}",
    MessageSql.message_persisted_ms <= cast(int(end_ms+10*60*1000), BigInteger),
    MessageSql.message_persisted_ms >= cast(int(start_ms-10*60*1000), BigInteger),
).order_by(asc(MessageSql.message_persisted_ms))

settings = Settings(_env_file=dotenv.find_dotenv())
engine = create_engine(settings.db_url_no_async.get_secret_value())
Session = sessionmaker(bind=engine)
session = Session()
result = session.execute(stmt)
messages = result.scalars().all()

flo_params_messages = []
for m in messages:
    if pendulum.from_timestamp(m.message_persisted_ms/1000, tz='America/New_York').minute == 57:
        print(f"Adding message from {m.from_alias} at {pendulum.from_timestamp(m.message_persisted_ms/1000, tz='America/New_York')}")
        flo_params_messages.append(m)

print(f"Found {len(messages)} messages and {len(flo_params_messages)} of them at minute 57\n")

session.close()
engine.dispose()

# ---------------------------------------------------
# Part 2: Find hourly data: hp_elec_in, hp_heat_out
# ---------------------------------------------------
# Use backoffice DB (same as get_electricity_use in visualizer_api.py)
gbo_engine = create_engine(settings.gbo_db_url_no_async.get_secret_value())
hourly_electricity = Table('hourly_electricity', MetaData(), autoload_with=gbo_engine)
GboSession = sessionmaker(bind=gbo_engine)
gbo_session = GboSession()
stmt_hourly = select(hourly_electricity).where(
    hourly_electricity.c.short_alias == house_alias,
    hourly_electricity.c.hour_start_s >= int(start_ms // 1000),
    hourly_electricity.c.hour_start_s <= int(end_ms // 1000),
).order_by(asc(hourly_electricity.c.hour_start_s))
hourly_records = gbo_session.execute(stmt_hourly).all()
# Build lists: one row per hour (hp_elec_in may be stored as hp_kwh_el in DB)
hourly_hour_start_s = []
hourly_hp_elec_in = []
hourly_hp_heat_out = []
for rec in hourly_records:
    hourly_hour_start_s.append(rec.hour_start_s)
    hourly_hp_elec_in.append(getattr(rec, 'hp_elec_in', getattr(rec, 'hp_kwh_el', 0)))
    hourly_hp_heat_out.append(getattr(rec, 'hp_kwh_th', getattr(rec, 'hp_heat_out', 0)))
print(f"Found {len(hourly_records)} hourly records for {house_alias} (hp_elec_in, hp_heat_out)")
gbo_session.close()
gbo_engine.dispose()

# ---------------------------------------------------
# Part 3: Generate plots in plots/ directory
# ---------------------------------------------------

if os.path.exists('plots'):
    shutil.rmtree('plots')
os.makedirs('plots', exist_ok=True)

true_init_energy, true_final_energy = [0]*len(flo_params_messages), [0]*len(flo_params_messages)
heat_to_store_expected = []
heat_from_hp_expected = []
true_initial_states, true_final_states = [None]*len(flo_params_messages), [None]*len(flo_params_messages)

for i, flo_params_msg in enumerate(flo_params_messages):
    flo_params = FloParamsHouse0(**flo_params_msg.payload)
    g = Flo(flo_params.to_bytes())
    g.solve_dijkstra()
    g.generate_recommendation(flo_params.to_bytes())
    v = DGraphVisualizer(g)
    v.plot(show=False,save_as=f'plots/flo{i+1}_graph.png')
    v.plot_pq_pairs(save_as=f'plots/flo{i+1}_pq_pairs.png')
    initial_node_edge: DEdge = [e for e in g.bid_edges[g.initial_node] if e.head == g.initial_node.next_node][0]
    hp_heat_out_expected = initial_node_edge.hp_heat_out
    heat_to_store_expected.append(hp_heat_out_expected - initial_node_edge.load_and_losses)
    heat_from_hp_expected.append(hp_heat_out_expected)

    winter_oak_supergraph_params = WinterOakSupergraphParams(
        num_layers=flo_params.num_layers,
        storage_volume_gallons=flo_params.storage_volume_gallons,
        hp_max_elec_kw=flo_params.hp_max_elec_kw,
        cop_intercept=flo_params.cop_intercept,
        cop_oat_coeff=flo_params.cop_oat_coeff,
        cop_min=flo_params.cop_min,
        cop_min_oat_f=flo_params.cop_min_oat_f,
        constant_delta_t=flo_params.constant_delta_t,
    )
    
    true_initial_node = DNode(
        top_temp=flo_params.initial_top_temp_f,
        middle_temp=flo_params.initial_middle_temp_f,
        bottom_temp=flo_params.initial_bottom_temp_f,
        thermocline1=flo_params.initial_thermocline_1,
        thermocline2=flo_params.initial_thermocline_2,
        parameters=winter_oak_supergraph_params,
    )
    true_initial_states[i] = true_initial_node

    true_init_energy[i] = true_initial_node.energy
    true_init_node = DNodeVisualizer(true_initial_node, 'true_initial')
    init_node = DNodeVisualizer(g.initial_node, 'initial')
    expected_node = DNodeVisualizer(g.initial_node.next_node, 'expected')
    true_init_node.plot(save_as=f'plots/flo{i+1}_true_initial.png')
    init_node.plot(save_as=f'plots/flo{i+1}_initial.png')
    expected_node.plot(save_as=f'plots/flo{i+1}_expected.png')
    if i!=0:
        true_final_node = DNode(
            top_temp=flo_params.initial_top_temp_f,
            middle_temp=flo_params.initial_middle_temp_f,
            bottom_temp=flo_params.initial_bottom_temp_f,
            thermocline1=flo_params.initial_thermocline_1,
            thermocline2=flo_params.initial_thermocline_2,
            parameters=winter_oak_supergraph_params,
        )
        true_final_states[i-1] = true_final_node
        true_final_energy[i-1] = true_final_node.energy
        true_final_node = DNodeVisualizer(true_final_node, 'true_final')
        true_final_node.plot(save_as=f'plots/flo{i}_true_final.png')
        final_node = DNodeVisualizer(g.initial_node, 'final')
        final_node.plot(save_as=f'plots/flo{i}_final.png')

    del g, v, init_node, expected_node
    gc.collect()

heat_to_store_true = [round(final-init, 2) for final, init in zip(true_final_energy, true_init_energy)]

# Save to local pickle files
import pickle
with open("true_initial_states.pkl", "wb") as f:
    pickle.dump(true_initial_states, f)
with open("true_final_states.pkl", "wb") as f:
    pickle.dump(true_final_states, f)
with open("heat_to_store_true.pkl", "wb") as f:
    pickle.dump(heat_to_store_true, f)
with open("flo_params_messages.pkl", "wb") as f:
    pickle.dump(flo_params_messages, f)


Welcome to the FLO report generator!

Generating report for oak from 2026-01-27 20:00:00-05:00 to 2026-01-31 07:00:00-05:00

Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 09:57:08.975000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 10:57:09.182000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 11:57:32.032000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 12:57:52.074000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 13:57:52.381000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 14:57:51.754000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 15:57:51.151000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 16:57:50.698000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 17:57:50.129000-05:00
Adding message from hw1.isone.me.versant.keene.oak at 2026-01-28 18:57:49.546000-05:00
Addi

2026-01-31 13:18:19,056 [INFO] gridflo.flo: Launching flo for hw1.isone.me.versant.keene.oak.scada
2026-01-31 13:18:19,057 [WARNING] gridflo.flo: glitch to hw1.isone.me.versant.keene.oak.scada: {'TypeName': 'glitch', 'Version': '001', 'FromGNodeAlias': 'hw1.isone.gwflo1', 'AboutGNodeAlias': 'hw1.isone.me.versant.keene.oak.scada', 'Level': 'Critical', 'Summary': 'Supergraph load failure', 'Details': "Supergraph meta data not found at  /Users/thomas/.config/gridworks/gridflo/supergraph_meta_380418a8.json with params {'TypeName': 'winter.oak.supergraph.params', 'Version': '001', 'NumLayers': 24, 'StorageVolumeGallons': 360, 'HpMaxElecKw': 11.0, 'CopIntercept': 1.02, 'CopOatCoeff': 0.0257, 'CopMin': 1.4, 'CopMinOatF': 15.0, 'ConstantDeltaT': 20}", 'CreatedMs': 1769861899057}


Found 52 hourly records for oak (hp_elec_in, hp_heat_out)


NeedsSupergraph: Supergraph meta data not found at  /Users/thomas/.config/gridworks/gridflo/supergraph_meta_380418a8.json with params {'TypeName': 'winter.oak.supergraph.params', 'Version': '001', 'NumLayers': 24, 'StorageVolumeGallons': 360, 'HpMaxElecKw': 11.0, 'CopIntercept': 1.02, 'CopOatCoeff': 0.0257, 'CopMin': 1.4, 'CopMinOatF': 15.0, 'ConstantDeltaT': 20}